In [1]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import itertools
from matplotlib.dates import date2num
import pandas as pd
import inspect
import datetime

import matplotlib
matplotlib.use('QT5Agg')


In [2]:
class OHLCAVWithDisaggCoT(btfeeds.GenericCSVData):
  
    lines = ('Tot_Rept_Positions_Long_All',
            'M_Money_Positions_Long_All',
            'Tot_Rept_Positions_Short_All',
            'M_Money_Positions_Short_All',)
    

    params = (
        ('nullvalue', 0.0),
        ('dtformat', ('%Y-%m-%d')),

        ('Date', None),
        ('Open', 1),
        ('High', 2),
        ('Low', 3),
        ('Close', 4),
        ('Adj Close', 5),
        ('Volume', 6),
        ('Tot_Rept_Positions_Long_All',7),
        ('Tot_Rept_Positions_Short_All',8),
        ('M_Money_Positions_Long_All',9),
        ('M_Money_Positions_Short_All',10),
    )
    
class PandasData_more(btfeeds.PandasData):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    lines = ('Tot_Rept_Positions_Long_All', 'Tot_Rept_Positions_Short_All',
             'M_Money_Positions_Long_All','M_Money_Positions_Short_All',)

    params = (
        ('Tot_Rept_Positions_Long_All',-1),
        ('Tot_Rept_Positions_Short_All',-1),
        ('M_Money_Positions_Long_All',-1),
        ('M_Money_Positions_Short_All',-1),
    )

In [3]:
class MyStrategy(bt.Strategy):
    params = (
            ('period', 15),
            ('printlog', False),
        )
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
    def __init__(self):
        print('init')
        print("--------- 打印 self.datas 第一个数据表格的 lines ----------")
        print(self.data0.lines.getlinealiases())
        # print("Type of self.data: {}".format(type(self.data)))
        # print(inspect.getmembers(self.data))
        self.sma = btind.SimpleMovingAverage(period=15)
        print("Type of self.sma: {}".format(type(self.sma)))
        # bt.talib.BBANDS(self.data)
        # print("Columns of self.data: {}".format(self.data.columns))
    
        self.M_Money_Long = btind.BollingerBands(self.data0.M_Money_Positions_Long_All, period=70, devfactor=1.5,plotname = 'Bollinger_M_Money_Long', subplot = True)
        self.M_Money_Short = btind.BollingerBands(self.data0.M_Money_Positions_Short_All, period=60, devfactor=1.5,plotname = 'Bollinger_M_Money_Short', subplot = True)
        # self.Fib = btind.FibonacciPivotPoint(self.data0.open, plotname = 'Fib', subplot = True)
        self.rsi2 = btind.RSI(self.data0.close, period=self.params.period - 10, plotname = 'RSI2', subplot = True)
        self.rsi12 = btind.RSI(self.data0.close, period = self.params.period, plotname = 'RSI12', subplot = True)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    def next(self):     
        # if rsi12 > 70 and rsi2 breaks rsi12, sell
        if self.rsi12[0] > 70 and self.rsi2[0] < self.rsi12[0]:
            if self.position.size > 0:
                self.close()
            else:
                self.sell()
        # if rsi12 < 30 and rsi2 breaks rsi12, buy
        if self.rsi12[0] < 30 and self.rsi2[0] > self.rsi12[0]:
            if self.position.size < 0:
                self.close()
            else:
                self.buy()

In [4]:
import collections
MAINSIGNALS = collections.OrderedDict(
    (('longshort', bt.SIGNAL_LONGSHORT),
     ('longonly', bt.SIGNAL_LONG),
     ('shortonly', bt.SIGNAL_SHORT),)
)


EXITSIGNALS = {
    'longexit': bt.SIGNAL_LONGEXIT,
    'shortexit': bt.SIGNAL_LONGEXIT,
}


class SMACloseSignal(bt.Indicator):
    lines = ('signal',)
    params = (('period', 30),)

    def __init__(self):
        self.lines.signal = self.data - bt.indicators.SMA(period=self.p.period)


class SMAExitSignal(bt.Indicator):
    lines = ('signal',)
    params = (('p1', 5), ('p2', 30),)

    def __init__(self):
        sma1 = bt.indicators.SMA(period=self.p.p1)
        sma2 = bt.indicators.SMA(period=self.p.p2)
        self.lines.signal = sma1 - sma2

In [5]:
dataname='../data/GC=F_com_disagg.csv'
# data = OHLCAVWithDisaggCoT(dataname=dataname)

cerebro = bt.Cerebro()
df = pd.read_csv(dataname, index_col=0)
df.index = pd.to_datetime(df.index)
# df


In [6]:
data = PandasData_more(dataname=df)

# data = OHLCAVWithDisaggCoT(dataname = dataname)

strats = cerebro.optstrategy(
        MyStrategy,
        period=range(15, 31))

cerebro.adddata(data)

# cerebro.addstrategy(MyStrategy)
# cerebro.add_signal(bt.SIGNAL_LONG, SMACloseSignal, period = 10)
# cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal)

# 初始资金 100,000,000
cerebro.broker.setcash(1e6)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addobserver(bt.observers.BuySell) # 买卖交易点

cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤


In [7]:
# 启动回测
result = cerebro.run()
cerebro.plot(iplot= False)
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())

